#infer

In [ ]:
#DESIGNATION	AGE	SALARY	PAST EXP		yearsMonths	UNIT_Finance	UNIT_IT	UNIT_Management	UNIT_Marketing	UNIT_Operations	UNIT_Web
selected_features = ['DESIGNATION',	'AGE','PAST EXP',	'yearsMonths','UNIT_Finance','UNIT_IT','UNIT_Management',	'UNIT_Marketing','UNIT_Operations','UNIT_Web']
target_feature = ['SALARY']

'DESIGNATION'

'Analyst'--> 0

'Senior Analyst' --> 1

'Associate' --> 2

'Manager'--> 3

'Senior Manager' --> 4

'Director' --> 5

UNIT_Finance	UNIT_IT	UNIT_Management	UNIT_Marketing	UNIT_Operations	UNIT_Web

'DESIGNATION'

'AGE'

'PAST EXP'

'yearsMonths'

'UNIT_Finance'

'UNIT_IT'

'UNIT_Management'

'UNIT_Marketing'

'UNIT_Operations'

'UNIT_Web'

AGE , SALARY --> log tranformation



read from user 'DESIGNATION', 'AGE', 'PAST EXP', 'yearsMonths', 'UNIT

pipeline

'DESIGNATION'

  'Analyst'--> 0

  'Senior Analyst' --> 1

  'Associate' --> 2

  'Manager'--> 3

  'Senior Manager' --> 4

  'Director' --> 5

AGE --> log

PAST EXP --> as it is

yearsMonths --> as it is

UNIT--> transform into 6 features

'UNIT_Finance'

'UNIT_IT'

'UNIT_Management'

'UNIT_Marketing'

'UNIT_Operations'

'UNIT_Web'


output --> salary log transformed , reverse it

In [114]:
%%writefile infer.py
import numpy as np

def pipeline(test_array):
  '''
  Function that receives the raw data from the user to process it to a format that the model can digest
    it perform label encoding to the designation, log transform the age, keeps the past_exp and years_months as they are, one-hot encode the unit

  Parameters:
    test_array (list): list of size 5 that contains the designation, age, past_exp, years_months, and unit respectively

  Return:
    features (list): a list of size 10 of the received list,test_array, after transformation
  '''
  DESIGNATION = ['Analyst','Senior Analyst','Associate','Manager','Senior Manager','Director'] # get index
  index = DESIGNATION.index(test_array[0])
  des = index

  age = np.log(test_array[1])

  past_exp = test_array[2]

  years_months = test_array[3]
  UNIT_CAT = ['Finance','IT','Management',	'Marketing','Operations','Web']
  UNITZeroes = [0,0,0,0,0,0]
  index = UNIT_CAT.index(test_array[4])
  UNITZeroes[index] = 1
  unit = UNITZeroes

  features  = [des, age, past_exp, years_months]
  features = features + unit
  return features

Writing infer.py


In [111]:
%cd /content/drive/MyDrive/machine_learning_projects/salary_prediction/salary-prediction

/content/drive/MyDrive/machine_learning_projects/salary_prediction/salary-prediction


In [112]:
%pwd

'/content/drive/MyDrive/machine_learning_projects/salary_prediction/salary-prediction'

In [113]:
%ls

 config.py        'Feature Engineering.ipynb'  'lazy prediction .ipynb'      __pycache__/
 custom_funcs.py  'Feature selection.ipynb'     LinearRegression.ipynb       README.md
 data/             functions.ipynb             'modeling .ipynb'             requirements.txt
 EDA.ipynb         git.ipynb                    models/                      salary-prediction/
 evaluate.ipynb    git_token.json              'pipeline processing.ipynb'   Untitled


In [128]:
%%writefile eval_pipeline.py
from custom_funcs import type_casting
from custom_funcs import convert_dates_to_duration
import numpy as np
from infer import pipeline
from sklearn import metrics

def testing(df):
  '''
  Function that receives the raw data as it is in the original test dataset and take only some of the features as selected from the feature selection step, transform the wrong datatypes, and add the new features add in the feature engineering step

  Parameters:
    df(DataFrame): raw test dataset as received

  Return:
    x(array of lists): tranformed dataset that is ready for evaluation
    y(array):
  '''
  x = df[['DOJ',	'CURRENT DATE',	'DESIGNATION',	'AGE',	'UNIT', 'PAST EXP']]
  y = df['SALARY']

  data_types_dict = {
    'int': ['AGE'],
    'datetime': ['DOJ','CURRENT DATE']
  }
  type_casting(x,data_types_dict)
  convert_dates_to_duration(x,'DOJ','CURRENT DATE')

  x = x[['DESIGNATION', 'AGE', 'PAST EXP', 'yearsMonths', 'UNIT']]

  x = x.apply(pipeline, axis = 1)
  y = np.log(y)
  return x,y


def print_evaluate(true, predicted):
    '''
    Functiom that prints the MAE, MSE, RMSE, and R2 score

    Parameters:
      true(series): a series that holds the true values or of the test dataset
      predicted(series): a series that holds the predected values or of the test dataset after prediction

    '''
    mae, mse, rmse, r2_square = evaluate(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')

def evaluate(true, predicted):
    '''
    Function that calculates the mean avg error, mean square error, root mean square error and r2 sqaure

    Parameters:
      true(series): a series that holds the true values or of the test dataset
      predicted(series): a series that holds the predected values or of the test dataset after prediction

    Return:
      mae(int): mean avg error
      mse(int): mean square error
      rmse(int): root mean square erro
      r2_square(int): r2 sqaure
    '''
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

Writing eval_pipeline.py


In [ ]:
import pandas as pd
row = pd.DataFrame(data = ('Senior Analyst', 21, 1.5, 12.6, 'Marketing'))

In [ ]:
row.T

,0,1,2,3,4
0,Senior Analyst,21,1.5,12.6,Marketing


In [34]:
ret = pipeline(['Senior Analyst', 21, 1.5, 12.6, 'Marketing'])
#1, 3.4011973816621555, 1.5, 12.6, 0, 0, 1, 0, 0

In [35]:
ret

[1, 3.044522437723423, 1.5, 12.6, 0, 0, 0, 1, 0, 0]

In [6]:
import pickle
model = pickle.load(open('/content/drive/MyDrive/machine_learning_projects/salary_prediction/salary-prediction/models/LeanearReg1.1.sav','rb'))

import pandas as pd

test = pd.read_csv('/content/drive/MyDrive/machine_learning_projects/salary_prediction/salary-prediction/data/test/X_test.csv')

In [8]:
test = test.dropna()

In [57]:
x,y =testing(test)

/content/drive/MyDrive/machine_learning_projects/salary_prediction/custom_funcs.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[data_types_dict['int']] = df[data_types_dict['int']].astype(int)
/content/drive/MyDrive/machine_learning_projects/salary_prediction/custom_funcs.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = pd.to_datetime(df[feature])
/content/drive/MyDrive/machine_learning_projects/salary_prediction/custom_funcs.py:147: SettingWithCopyWarning: 
A value is trying to 

In [59]:
x

0      [1, 3.332204510175204, 3, 2.4166666666666665, ...
1      [2, 3.4339872044851463, 5, 1.5, 0, 0, 0, 1, 0, 0]
2      [0, 3.1354942159291497, 0, 2.3333333333333335,...
3      [0, 3.1780538303479458, 1, 1.6666666666666665,...
4      [0, 3.2188758248682006, 2, 2.0833333333333335,...
                             ...                        
523    [0, 3.1354942159291497, 1, 1.1666666666666667,...
524    [1, 3.295836866004329, 2, 3.5833333333333335, ...
525    [0, 3.091042453358316, 0, 1.8333333333333335, ...
526    [0, 3.091042453358316, 0, 1.75, 0, 0, 0, 0, 0, 1]
527     [1, 3.332204510175204, 3, 4.0, 0, 0, 0, 0, 0, 1]
Length: 524, dtype: object

In [38]:
y

0      10.872181
1      11.377381
2      10.705444
3      10.643041
4      10.670675
         ...    
523    10.800636
524    11.005527
525    10.722474
526    10.702750
527    11.025377
Name: SALARY, Length: 524, dtype: float64

In [15]:
type(x.values.tolist())

list

In [48]:
def pr(arr):
  return model.predict(np.array(arr).reshape(1,-1))

In [62]:
y_pred = x.apply(pr)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature n

In [51]:
type(y)

pandas.core.series.Series

In [52]:
from sklearn.metrics import r2_score
r2 = r2_score(y, y_pred)

In [64]:
r2

0.9478234217946953

In [54]:
from sklearn import metrics

def print_evaluate(true, predicted):
    mae, mse, rmse, r2_square = evaluate(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')

def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [55]:
test_pred = y_pred

print('Test set evaluation:\n_____________________________________')
print_evaluate(y, test_pred)

Test set evaluation:
_____________________________________
MAE: 0.06749809805980801
MSE: 0.006655037272039222
RMSE: 0.08157841180140259
R2 Square 0.9478234217946953
__________________________________


In [127]:
type(y_pred)

pandas.core.series.Series

In [ ]:
y_pred

0      [11.033331894099842]
1         [11.379018574727]
2      [10.699433757530088]
3      [10.697279432113646]
4      [10.698225585483573]
               ...         
523    [10.703827642634733]
524    [11.035912556360287]
525     [10.71241799937193]
526     [10.71208567019365]
527    [11.039965755746607]
Length: 524, dtype: object

In [82]:
exp_pred = np.exp(y_pred.tolist())

In [ ]:
exp_pred.reshape(-1)

In [124]:
exp_y = np.exp(y)

In [125]:
exp_y

0      52690.0
1      87324.0
2      44598.0
3      41900.0
4      43074.0
        ...   
523    49052.0
524    60206.0
525    45364.0
526    44478.0
527    61413.0
Name: SALARY, Length: 524, dtype: float64

In [101]:
print_evaluate(exp_y,exp_pred)

MAE: 4212.470168029973
MSE: 46618962.26347271
RMSE: 6827.808013079506
R2 Square 0.9619996199584748
__________________________________
